In [171]:
# Dependencies and Setup
import pandas as pd

In [172]:
schools_complete = "Resources/schools_complete.csv"
students_complete = "Resources/students_complete.csv"

In [173]:
schools_complete_df = pd.read_csv(schools_complete)
students_complete_df = pd.read_csv(students_complete)


In [174]:
# calculate per student and correct per school budget
schools_complete_df["Per student budget"] = schools_complete_df["budget"] / schools_complete_df["size"]
# schools_complete2_df = schools_complete_df.sort_values(["budget"], ascending=True)
# schools_complete_df

In [175]:
# The following was used to calculate stats of per student budget
# schools_complete_df["Per student budget"].describe()
# Results are
# count     15.000000
# mean     620.066667
# std       28.544368
# min      578.000000
# 25%      591.500000
# 50%      628.000000
# 75%      641.500000
# max      655.000000
# Name: Per student budget, dtype: float64

# Will use the 4 quartiles for Scores by School Spending
bins1 = [578, 591, 628, 641, 655]
group_names1 = ["$578-591", "$628-591", "$641-628", "$655-641"]
pd.cut(schools_complete_df["Per student budget"], bins1, labels=group_names1).head()
students_complete_df["Spending Ranges (Per Student)"] = pd.cut(schools_complete_df["Per student budget"], bins1, labels=group_names1)
# students_complete_df.head()

In [176]:
# The following was used to calculate stats of per student budget
# schools_complete_df["size"].describe()
# count      15.000000
# mean     2611.333333
# std      1420.915282
# min       427.000000
# 25%      1698.000000
# 50%      2283.000000
# 75%      3474.000000
# max      4976.000000
# Name: size, dtype: float64
# Will use the 4 quartiles for Scores by School Spending
# bins2 = [427, 1698, 2283, 3474, 4976]
# group_names2 = ["427-1698", "2283-1698", "3474-2283", "4976-3474"]
bins2 = [427, 999, 1999, 4976]
group_names2 = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]
pd.cut(schools_complete_df["size"], bins2, labels=group_names2).head()
students_complete_df["Scores by School Size"] = pd.cut(schools_complete_df["size"], bins2, labels=group_names2)
# students_complete_df.head()

In [177]:
# add % Passing Maths	% Passing Reading
bins = [0, 49.9, 100]
group_names = [0, 1]
pd.cut(students_complete_df["maths_score"], bins, labels=group_names).head()
pd.cut(students_complete_df["reading_score"], bins, labels=group_names).head()
students_complete_df["maths_result"] = pd.cut(students_complete_df["maths_score"], bins, labels=group_names)
students_complete_df["maths_result"] = students_complete_df["maths_result"].astype(float)
students_complete_df["reading_result"] = pd.cut(students_complete_df["reading_score"], bins, labels=group_names)
students_complete_df["reading_result"] = students_complete_df["reading_result"].astype(float)

In [178]:
# add % Overall Passing
students_complete_df.loc[(students_complete_df['maths_result'] + students_complete_df['reading_result']) > 1, 'total_result'] = '1' 
students_complete_df.loc[(students_complete_df['maths_result'] + students_complete_df['reading_result']) <= 1, 'total_result'] = '0' 
students_complete_df["total_result"] = students_complete_df["total_result"].astype(float)
students_complete_df.loc[(students_complete_df["year"]) == 9, 'Year 9 M'] = (students_complete_df["maths_score"]) 
students_complete_df.loc[(students_complete_df["year"]) == 10, 'Year 10 M'] = (students_complete_df["maths_score"]) 
students_complete_df.loc[(students_complete_df["year"]) == 11, 'Year 11 M'] = (students_complete_df["maths_score"]) 
students_complete_df.loc[(students_complete_df["year"]) == 12, 'Year 12 M'] = (students_complete_df["maths_score"]) 
students_complete_df.loc[(students_complete_df["year"]) == 9, 'Year 9 R'] = (students_complete_df["reading_score"]) 
students_complete_df.loc[(students_complete_df["year"]) == 10, 'Year 10 R'] = (students_complete_df["reading_score"]) 
students_complete_df.loc[(students_complete_df["year"]) == 11, 'Year 11 R'] = (students_complete_df["reading_score"]) 
students_complete_df.loc[(students_complete_df["year"]) == 12, 'Year 12 R'] = (students_complete_df["reading_score"]) 
# students_complete_df

In [179]:
# create groupby of students
student_grouped_data = students_complete_df.groupby(["school_name"])
student_grouped_data

In [180]:
# create mean table
student_grouped_mean_df = student_grouped_data.mean()
# student_grouped_mean_df

In [181]:
# Maths results per year
math_result_year_df = student_grouped_mean_df[["Year 9 M", "Year 10 M", "Year 11 M", "Year 12 M"]]
# math_result_year_df

In [182]:
# Reading results per year
reading_result_year_df = student_grouped_mean_df[["Year 9 R", "Year 10 R", "Year 11 R", "Year 12 R"]]
# reading_result_year_df

In [183]:
# merge DFs
merged_school_data_df = pd.merge(student_grouped_mean_df, schools_complete_df, how="left", on=["school_name", "school_name"])
# my original code: merged_school_data_df = pd.merge(schools_complete_df, student_grouped_mean_df, on="school_name")
merged_school_data_df.head()

,school_name,Student ID,year,reading_score,maths_score,maths_result,reading_result,total_result,Year 9 M,Year 10 M,...,Year 12 M,Year 9 R,Year 10 R,Year 11 R,Year 12 R,School ID,type,size,budget,Per student budget
0,Bailey High School,20358.5,10.371584,71.008842,72.352894,0.916399,0.873794,0.800844,72.493827,71.897498,...,72.675097,70.901920,70.848265,70.317346,72.195525,7,Government,4976,3124928,628.0
1,Cabrera High School,16941.5,10.386437,71.359526,71.657158,0.908504,0.890743,0.807858,72.321970,72.437768,...,70.604712,71.172348,71.328326,71.201245,71.856021,6,Independent,1858,1081356,582.0
2,Figueroa High School,4391.0,10.371312,69.077993,68.698542,0.816548,0.828077,0.676501,68.477804,68.331586,...,69.325282,70.261682,67.677588,69.152327,69.082126,1,Government,2949,1884411,639.0
3,Ford High School,36165.0,10.330047,69.572472,69.091274,0.824388,0.822198,0.674699,69.021609,69.387006,...,68.617811,69.615846,68.988701,70.735964,68.849722,13,Government,2739,1763916,644.0
4,Griffin High School,12995.5,10.365123,71.245232,71.788147,0.912125,0.884877,0.813351,72.789731,71.093596,...,71.469178,72.026895,70.746305,72.385042,69.434932,4,Independent,1468,917500,625.0


In [184]:
# select columns
merged_school_data_ltd_df = merged_school_data_df[["school_name","type", "size", "budget", "Per student budget", "reading_score", "maths_score", "maths_result", "reading_result", "total_result"]]
merged_school_data_ltd_df.head()

,school_name,type,size,budget,Per student budget,reading_score,maths_score,maths_result,reading_result,total_result
0,Bailey High School,Government,4976,3124928,628.0,71.008842,72.352894,0.916399,0.873794,0.800844
1,Cabrera High School,Independent,1858,1081356,582.0,71.359526,71.657158,0.908504,0.890743,0.807858
2,Figueroa High School,Government,2949,1884411,639.0,69.077993,68.698542,0.816548,0.828077,0.676501
3,Ford High School,Government,2739,1763916,644.0,69.572472,69.091274,0.824388,0.822198,0.674699
4,Griffin High School,Independent,1468,917500,625.0,71.245232,71.788147,0.912125,0.884877,0.813351


In [185]:
student_grouped_data2 = merged_school_data_ltd_df.groupby(["school_name"])
student_grouped_data2
student_grouped_data2_df = student_grouped_data2.mean()
# # Scores by School Type
# merged_school_data_df = pd.merge(student_grouped_mean_df, schools_complete_df, how="left", on=["school_name", "school_name"])
# # my original code: merged_school_data_df = pd.merge(schools_complete_df, student_grouped_mean_df, on="school_name")
# merged_school_data_df.head()student_grouped_data2_df.head()

In [187]:
schooltype_grouped_data = merged_school_data_ltd_df.groupby(["type"])
schooltype_grouped_data
schooltype_grouped_data_df = schooltype_grouped_data.mean()
schooltype_grouped_data_df.head()

,size,budget,Per student budget,reading_score,maths_score,maths_result,reading_result,total_result
type,,,,,,,,
Government,3853.714286,2.478275e+06,643.571429,69.675929,69.834806,0.844624,0.835876,0.706990
Independent,1524.250000,9.126881e+05,599.500000,70.718933,71.368822,0.892040,0.862478,0.769733


In [190]:
# # Scores by School size

schoolsize_merged_data_df = pd.merge(students_complete_df,schools_complete_df, how="left", on=["school_name", "school_name"])
# my original code: merged_school_data_df = pd.merge(schools_complete_df, student_grouped_mean_df, on="school_name")
schooltype_grouped_data_df.head()

# merged_school_data_df = pd.merge(student_grouped_mean_df, schools_complete_df, how="left", on=["school_name", "school_name"])
# # my original code: merged_school_data_df = pd.merge(schools_complete_df, student_grouped_mean_df, on="school_name")
# merged_school_data_df.head()

schoolsize_grouped_data = schoolsize_merged_data_df.groupby(["Scores by School Size"])
schoolsize_grouped_data
schoolsize_grouped_data_df = schoolsize_grouped_data.mean()
schoolsize_grouped_data_df.head()

,Student ID,year,reading_score,maths_score,maths_result,reading_result,total_result,Year 9 M,Year 10 M,Year 11 M,Year 12 M,Year 9 R,Year 10 R,Year 11 R,Year 12 R,School ID,size,budget,Per student budget
Scores by School Size,,,,,,,,,,,,,,,,,,,
Small (<1000),9.0,10.00,64.000,77.00,1.000,1.0,1.000,NaN,77.0,NaN,NaN,NaN,64.0,NaN,NaN,0.0,2917.0,1910635.0,655.0
Medium (1000-2000),7.2,10.40,66.200,70.00,1.000,0.8,0.800,69.0,72.5,60.0,76.000000,87.00,65.0,73.0,41.000000,0.0,2917.0,1910635.0,655.0
Large (2000-5000),6.5,10.25,80.875,79.75,0.875,1.0,0.875,89.0,89.0,NaN,64.333333,83.25,66.0,NaN,82.666667,0.0,2917.0,1910635.0,655.0


In [ ]:
merged_schoolsize_df = pd.merge(student_grouped_mean_df, schools_complete_df, how="left", on=["school_name", "school_name"])

In [121]:
# student_grouped_data2_df["budget"] = student_grouped_data2_df["budget"].astype(float).map("${:,.2f}".format)
# student_grouped_data2_df["maths_result"] = student_grouped_data2_df["maths_result"].astype(float).map("{0:.2f}%".format)
# student_grouped_data2_df.head()

In [122]:
top_performing_schools_df = student_grouped_data2.sort_values(["total_result"], ascending=False)
top_performing_schools_df.head()

AttributeError: 'DataFrameGroupBy' object has no attribute 'sort_values'

In [123]:
worst_performing_schools_df = student_grouped_data2.sort_values(["total_result"], ascending=False)
worst_performing_schools_df.head()

AttributeError: 'DataFrameGroupBy' object has no attribute 'sort_values'

In [ ]:
# create count table
school_count_df = grouped_data.count()

In [ ]:
# merge sum and count
merged_sum_count_df = pd.merge(school_sum_df["budget"], school_count_df, on="school_name"

In [ ]:
# maths scores by year
students_complete_df.loc[(students_complete_df["Year"]) == 9, 'Year 9'] = (students_complete_df["maths_score"]) 
students_complete_df.loc[(students_complete_df['maths_result'] + students_complete_df['reading_result']) <= 1, 'total_result'] = '0' 
students_complete_df["total_result"] = students_complete_df["total_result"].astype(float)
# students_complete_df